# Into to NNCME with a Toggle Switch Example

This is a detailed step-by-step guideline of implementing a toggle switch example with NNCME. \
A simple example of a gene expression system can be seen in `Start with a Simple Gene Expression Example.ipynb`.\
If you would like to explore more with NNCME, please refer to the `.py` files of other representative examples.


## 1. Toggle Switch System
The toggle switch system involves 6 molecular species:  genes $G_{x}$, $G_{y}$, which transcript two proteins $P_{x}$, $P_{y}$ inhibiting the gene expression of each other. There are two protein-DNA complexes $\bar{G}_{x}$, $\bar{G}_{y}$, with protein $P_{y}$ ($P_{x}$) bound to gene $G_{x}$ ($G_{y}$). The dimer of the protein $P_{x}$ ($P_{y}$) inhibits the activity of the gene $G_{y}$ ($G_{x}$). 

<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_panela.svg" width="200"/> <br>
</td> 
</table>

The model can be written as the chemical reactions:
\begin{align}
\begin{split}
G_{x}\stackrel{s_{x}}{\longrightarrow}G_{x}+P_{x},\quad
G_{y}\stackrel{s_{y}}{\longrightarrow}G_{y} & +P_{y},\quad
P_{x}\stackrel{d_{x}}{\longrightarrow}\emptyset,\quad
P_{y}\stackrel{d_{y}}{\longrightarrow}\emptyset, \\
2P_{x}+G_{y}\stackrel{b_{y}}{\longrightarrow}\bar{G}_{y},\quad
2P_{y}+G_{x}\stackrel{b_{x}}{\longrightarrow}\bar{G}_{x},\quad
&\bar{G}_{y}\stackrel{u_{y}}{\longrightarrow}2P_{x}+G_{y},\quad
\bar{G}_{x}\stackrel{u_{x}}{\longrightarrow}2P_{y}+G_{x},
\end{split}
\end{align}
where $s_{x}$, $s_{y}$ are the synthesis rates of the proteins, and $p_{x}$, $p_{y}$ are the degradation rates of the proteins. 
The transition rate $b_{y}$ ($b_{x}$) is the binding rate of two copies of protein $P_{X}$ ($P_{Y}$) to the $G_{y}$ ($G_{x}$), to form the complex $\bar{G}_{y}$ ($\bar{G}_{x}$). The unbinding of the complex $\bar{G}_{y}$ ($\bar{G}_{x}$) has the rate $u_{y}$ ($u_{x}$).  

The stoichiometeric matrix can be written as with row $i$ represents species $i$ and column $k$ represents reaction $j$ (the order of species here is $G_{x}$,$G_{y}$,$P_{X}$,$P_{Y}$,$\bar{G}_{x}$,$\bar{G}_{y}$）

$$V=
\begin{bmatrix} 
    0 & 0 & 0 & 0 & 0 & -1 & 0 & 1 \\ 
    0 & 0 & 0 & 0 & -1 & 0 & 1 & 0 \\
    1 & 0 & -1 & 0 & -2 & 0 & 2 & 0 \\
    0 & 1 & 0 & -1 & 0 & -2 & 0 & 2 \\
    0 & 0 & 0 & 0 & 0 & 1 & 0 & -1 \\
    0 & 0 & 0 & 0 & 1 & 0 & -1 & 0
\end{bmatrix}.
$$

The concentrations of each species are given by the following ODEs: 

\begin{gather}
\frac{dG_{x}}{dt}=-b_{x}P_{y}(P_{y}-1)G_{x}+u_{x}\bar{G}_{x}\\
\frac{dG_{y}}{dt}=-b_{y}P_{x}(P_{x}-1)G_{y}+u_{y}\bar{G}_{y}\\
\frac{dP_{x}}{dt}=s_{x}G_{x}-d_{x}P_{x}-2b_{x}P_{y}(P_{y}-1)G_{x}+2u_{y}\bar{G}_{y}\\
\frac{dP_{y}}{dt}=s_{y}G_{y}-d_{y}P_{y}-2b_{y}P_{x}(P_{x}-1)G_{y}+2u_{x}\bar{G}_{x}\\
\frac{d\bar{G}_{x}}{dt}=b_{x}P_{y}(P_{y}-1)G_{x}-u_{x}\bar{G}_{x}\\
\frac{d\bar{G}_{y}}{dt}=b_{y}P_{x}(P_{x}-1)G_{y}-u_{y}\bar{G}_{y}
\end{gather}

The total count of the two forms for each gene is conserved: $G_{x}+\bar{G}_{x}=1$. This conservation sets a constraint on the count of the two genes: $G_{x}=0, 1$, $G_{y}=0, 1$, and effectively reduces two variables by  $\bar{G}_{x}=1-G_{x}$, $\bar{G}_{y}=1-G_{y}$. Therefore, in the following code, the number of species is set 4.

## 2. Create a ToggleSwitch.py
Input toggle switch system as a `.py` file. Include initial conditions, species number constraint, reaction rates and the stoichiometric matrix in the function 'rates' . What's more, function 'init' and 'Propensity' are essential for every system and function 'MaskToggleSwitch' is used to constrain the count of certain species in this system.

In [ ]:
###ToggleSwitch.py
import numpy as np
import torch

class ToggleSwitch:
    
    def rates(self): 

        self.L=4 # Number of spscies
        IniDistri='delta' # Choose delta distribution to generate initial distribution
        initialD=np.array([1,1,0,0]).reshape(1,self.L) # The parameter for the initial delta distribution
        # here we set G_x=1, G_y=1, P_x=0, P_y=0
        
        MConstrain=np.array([2,2,self.M,self.M], dtype=int) # number constraint (not include the upper limit)
        # here G_x,G_y=0,1 therefore set G_x<2, G_y<2 
        # Add different number constraint for each species. If the upper limits for all species are all the same as self.M, then set Mconstrain=np.zeros(1,dtype=int)
        conservation=np.ones(1,dtype=int) # To maintain the total number of all species, set conservation=initial_num.sum(), otherwise set conservation=np.ones(1,dtype=int).
    
        r=torch.zeros(8) # reaction rates, 8 is the reation number in this system
        sx=sy=50
        dx=dy=1
        by=bx=1e-4
        uy=ux=0.1
        
        r[0] = sx
        r[1] = sy
        r[2] = dx
        r[3] = dy
        r[4] = by
        r[5] = bx
        r[6] = uy
        r[7] = ux

        # Reaction matrix
        ReactionMatLeft=torch.as_tensor([(1,0,0,0,0,1,0,0),(0,1,0,0,1,0,0,0),(0,0,1,0,2,0,0,0),(0,0,0,1,0,2,0,0)]).to(self.device)#SpeciesXReactions
        #The reaction matrix on the left (dimension: species_num*reactions_num) (stoichiometric matrix = reaction_matrix_left-reaction_matrix_right) 
        ReactionMatRight=torch.as_tensor([(1,0,0,0,0,0,0,1),(0,1,0,0,0,0,1,0),(1,0,0,0,0,0,2,0),(0,1,0,0,0,0,0,2)]).to(self.device)#SpeciesXReactions
        #The reaction matrix on the right (dimension: species_num*reactions_num) (stoichiometric matrix = reaction_matrix_left-reaction_matrix_right) 
            
        return IniDistri,initialD,r,ReactionMatLeft,ReactionMatRight,MConstrain,conservation
    
    def __init__(self, *args, **kwargs):
        super().__init__()
        #self.n = kwargs['n']
        self.L = kwargs['L']
        self.M = kwargs['M']
        self.bits = kwargs['bits']  
        self.device = kwargs['device']
        self.MConstrain = kwargs['MConstrain']
        # self.Para = kwargs['Para']
        self.IniDistri = kwargs['IniDistri']
    
    # It is used to constrain the count of certain species. 
    # For example, DNA typically only has the count of 0 or 1 inside a cell. 
    # The "Mask" function allows only the reactions with such a proper count to occur.
    def MaskToggleSwitch(self,SampleNeighbor1D_Win,WinProd):
        Mask1=torch.ones_like(WinProd)
        Gx=SampleNeighbor1D_Win[:,0,:] #Gx for different reactions: the second dimension of SampleNeighbor1D_Win is the label of species
        Gy=SampleNeighbor1D_Win[:,1,:] #Gy for different reactions: the second dimension of SampleNeighbor1D_Win is the label of species
        Gx1=1-Gx
        Gy1=1-Gy
        Mask1[Gx[:,0]!=1,0]=0 #The second dimension of Gx and Mask1 is the label of reactions
        Mask1[Gx[:,5]!=1,5]=0
        Mask1[Gy[:,1]!=1,1]=0
        Mask1[Gy[:,4]!=1,4]=0
        Mask1[Gx1[:,7]!=1,7]=0
        Mask1[Gy1[:,6]!=1,6]=0
        
        return Mask1
    
    def Propensity(self,Win,Wout,cc,SampleNeighbor1D_Win,SampleNeighbor1D,NotHappen_in_low,NotHappen_in_up,NotHappen_out_low,NotHappen_out_up):
        WinProd=torch.prod(Win,1)
        Mask1=self.MaskToggleSwitch(SampleNeighbor1D_Win,WinProd)
        Propensity_in=WinProd*Mask1*cc
        WoutProd=torch.prod(Wout,1)
        Mask1=self.MaskToggleSwitch(SampleNeighbor1D,WoutProd)
        Propensity_out=WoutProd*Mask1*cc
        
        return Propensity_in,Propensity_out

## 3. Implement the code after providing the ToggleSwitch.py file

(a) Server: Use a `ToggleSwitch.sh` to input the hyperparameters from the shell. 

In [ ]:
#!/bin/bash
#SBATCH --job-name=ToggleSwitch-rnn-1-32-M80-Tstep8001-dt0.005
#SBATCH --ntasks=1
#SBATCH --nodes=1
#SBATCH --partition=v100
#SBATCH --gres=gpu:1
#SBATCH -o out/%j.out
#SBATCH -e out/%j.out
#SBATCH -t 13-2:00:00

# load the environment
module purge

CUDA_LAUNCH_BLOCKING=1 python3 MasterEq.py --L 4 --M 80 --Model 'ToggleSwitch' --net 'rnn' --lossType 'kl' --max_stepAll 5000 --max_stepLater 100 --lr 0.001 --net_depth 1 --net_width 32 --print_step 40 --batch_size 1000 --Tstep 8001 --delta_t 0.005 --cuda 0 --dtype float64

#exit 0


(b) PC users (Windows): Run `MasterEq.py` after necessary changes according to the toggle switch example (importing the model class, adjusting the hyperparameters and adding model command).

In [ ]:
###MasterEq.py
from args import args
import numpy as np
from main import Test

###Add models----------------------------------
from ToggleSwitch import ToggleSwitch  #from ToggleSwitch.py file import the ToggleSwitch class
from EarlyLife import EarlyLife
from Epidemic import Epidemic
from cascade1 import cascade1
from cascade1_inverse import cascade1_inverse
from cascade2 import cascade2
from cascade3 import cascade3
from BirthDeath import BirthDeath
from GeneExp import GeneExp
from AFL import AFL

##Set parameters-------------------------------
###Initialize parameters: otherwise the parameters are specified in init_out_dir-> args.py
args.Model='ToggleSwitch' #Model name
args.L=4#Species number
args.M=int(80) #Upper limit of the molecule number: it is adjustable and can be indicated by doing a few Gillespie simulation. 
args.batch_size=1000 #Number of batch samples
args.Tstep=8001# Time step of iterating the chemical master equation
args.delta_t=0.0005 #Time step length of iterating the chemical master equation, depending on the reaction rates

args.net ='rnn'
args.max_stepAll=5000 #Maximum number of steps first time step (usually larger to ensure the accuracy)
args.max_stepLater=100 #Maximum number of steps of later time steps
args.net_depth=1 # including output layer and not input data layer
args.net_width=32
args.d_model=16# transformer
args.d_ff=32# transformer
args.n_layers=2# transformer
args.n_heads=2# transformer
args.lr=0.001
args.binary=False
args.AdaptiveT=False
args.AdaptiveTFold=5
args.print_step=20
args.saving_data_time_step=[0,1e2,5e2,2e3,1e4,2e4,5e4,1e5,1.5e5,2e5,2.5e5,3e5,3.5e5,4e5,5e5,6e5,7e5,8e5,9e5,1e6] #To save data at which time steps (give in a list)
args.training_loss_print_step=[0,1,2,101,1001,2e3,1e4,1e5,2e5,3e5,4e5,5e5] #To print training loss at which time steps (give in a list)

###Default parameters:
args.bias=True
args.bits=1
if args.binary:
    args.bits=int(np.ceil(np.log2(args.M)))
args.Percent=0.2         
args.clip_grad=1
args.dtype='float64'
args.epsilon=1e-30#initial probability of zero species number
args.lr_schedule=False#True

###Add model command-------------------------------
if args.Model=='ToggleSwitch':
    model = ToggleSwitch(**vars(args))   
if args.Model=='EarlyLife':
    model = EarlyLife(**vars(args))   
if args.Model=='Epidemic':
    model = Epidemic(**vars(args))   
if args.Model=='cascade1': 
    model = cascade1(**vars(args)) 
if args.Model=='cascade1_inverse':
    model = cascade1_inverse(**vars(args)) 
if args.Model=='cascade2':
    model = cascade2(**vars(args))    
if args.Model=='cascade3':
    model = cascade3(**vars(args))    
if args.Model=='BirthDeath':
    model = BirthDeath(**vars(args))   
if args.Model=='GeneExp':
    model = GeneExp(**vars(args))   
if args.Model=='AFL':
    model = AFL(**vars(args)) 
    
#Run model-----------------------------------------        
if __name__ == '__main__':
    Test(model)    


## 4. Optional: Run Gillespie simulation
Gillespie algorithm, also known as the stochastic simulation algorithm or the kinetic Monte Carlo, simulates trajectories to generate statistics of relevant variables. To evaluate the accuracy of the learnt distribution by the VAN, we can compare the resultant marginal distribution of one species with those from Gillespie algorithm.

In [ ]:
import numpy as np
import biocircuits
import matplotlib.pyplot as plt

def ToggleSwitch_propensity(
    propensities, population, t, sx, sy, dx, dy, by, bx, uy, ux
):
    #species
    Gx, Gy, Px, Py = population  
    Gx1=1-Gx
    Gy1=1-Gy
    #propensities
    propensities[0] = sx*Gx
    propensities[1] = sy*Gy
    propensities[2] = dx*Px
    propensities[3] = dy*Py
    propensities[4] = by*Px*Px*Gy
    propensities[5] = bx*Py*Py*Gx
    propensities[6] = uy*Gy1
    propensities[7] = ux*Gx1

#the stoichiometric matrix   
ToggleSwitch_update = np.array(
    [
        [ 0, 0, 1, 0], 
        [ 0, 0, 0, 1],
        [ 0, 0,-1, 0],
        [ 0, 0, 0,-1], 
        [ 0,-1,-2, 0], 
        [-1, 0, 0,-2],
        [ 0, 1, 2, 0],
        [ 1, 0, 0, 2]
    ],dtype=int)

#the reaction rates 
sx=sy=50
dx=dy=1
by=bx=1e-4
uy=ux=0.1

ToggleSwitch_args = (sx, sy, dx, dy, by, bx, uy, ux)

#initial number of species 
Gx0=0
Gy0=0
Px0=0
Py0=0
ToggleSwitch_pop_0 = np.array([Gx0,Gy0,Px0,Py0], dtype=float) # follow VAN's learnt initial number

#simulation time length
T=40
time_points = np.linspace(0, T, T+1)

In [ ]:
Run=1 #run Gillespie or not
times=1000 #Gillespie simulation times

out_filename = 'ToggleSwitch_times'+str(times)+'_T'+str(T)+'_dis'+str(Gx0)+'_'+str(Gy0)+str(Px0)+'_'+str(Py0) #filename to save
if Run==1:

    Gx_total=np.empty(shape=(0,len(time_points)))#to save the time evolution of Gx number in each simulation (dimension: times*time_points)
    Gx1_total=np.empty(shape=(0,len(time_points)))#to save the time evolution of \bar{Gx} number in each simulation (dimension: times*time_points)
    Gy_total=np.empty(shape=(0,len(time_points)))#to save the time evolution of Gy number in each simulation (dimension: times*time_points)
    Gy1_total=np.empty(shape=(0,len(time_points)))#to save the time evolution of \bar{Gy} number in each simulation (dimension: times*time_points)
    Px_total=np.empty(shape=(0,len(time_points)))#to save the time evolution of Px number in each simulation (dimension: times*time_points)
    Py_total=np.empty(shape=(0,len(time_points)))#to save the time evolution of Py number in each simulation (dimension: times*time_points)

    for i in range(times):
        
        # Perform the Gillespie simulation
        pop = biocircuits.gillespie_ssa(
            ToggleSwitch_propensity,
            ToggleSwitch_update,
            ToggleSwitch_pop_0,
            time_points,
            args=ToggleSwitch_args,
        )
        
        Gx_total=np.row_stack((Gx_total,pop[0,:,0]))
        Gx1_total=np.row_stack((Gx1_total,1-pop[0,:,0]))
        Gy_total=np.row_stack((Gy_total,pop[0,:,1]))
        Gy1_total=np.row_stack((Gy1_total,1-pop[0,:,1]))
        Px_total=np.row_stack((Px_total,pop[0,:,2]))
        Py_total=np.row_stack((Py_total,pop[0,:,3]))

        if i<3: #Plot fist three simulations
            plt.rc('font', size=16)
            plt.figure()
            plt.plot(time_points ,pop[0,:,0])
            plt.plot(time_points ,pop[0,:,1])
            plt.plot(time_points ,pop[0,:,2])
            plt.plot(time_points ,pop[0,:,3])            
            plt.xlabel("time")
            plt.ylabel("species number")
            plt.grid()
            plt.show()
        
    np.savez('{}'.format(out_filename),np.array(times),np.array(time_points),Gx_total,Gx1_total,Gy_total,Gy1_total,Px_total,Py_total) #save data
else: #load existing data file
    data=np.load(out_filename+'.npz')
    print(list(data))    
    time_points = data['arr_1']
    Gx_total = data['arr_2']
    Gx1_total = data['arr_3']
    Gy_total = data['arr_4']
    Gy1_total = data['arr_5']
    Px_total = data['arr_6']
    Py_total = data['arr_7']

In [ ]:
#Plot Gillespie Result
plt.rc('font', size=16)
plt.figure(num=None,  dpi=400, edgecolor='k')
fig, axes = plt.subplots(2,1)
fig.tight_layout()
ax = plt.subplot(1,1, 1)
from matplotlib import cm
jet = cm.get_cmap('jet')
Number=9
jet_12_colors = jet(np.linspace(0, 1, Number))
for Species in np.arange(4):
    if Species==0:
        plt.errorbar(time_points, np.mean(Gx_total,0),  yerr=np.std(Gx_total,0), color=jet_12_colors[Species*2+1,:])    
    if Species==1:
        plt.errorbar(time_points, np.mean(Gy_total,0),  yerr=np.std(Gy_total,0), color=jet_12_colors[Species*2+1,:])    
    if Species==2:
        plt.errorbar(time_points, np.mean(Px_total,0),  yerr=np.std(Px_total,0), color=jet_12_colors[Species*2+1,:])  
    if Species==3:
        plt.errorbar(time_points, np.mean(Py_total,0),  yerr=np.std(Py_total,0), color=jet_12_colors[Species*2+1,:])  

plt.xlabel('Time')
plt.ylabel('Species Number')
plt.legend()
fig.set_size_inches(9, 8)
plt.figure()

## 5. Plot Results
Plot the result of the genetic toggle switch. Relevant details can be referred in the manuscript.

In [ ]:
# load data of the VAN and Gillespie
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import pandas as pd
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib as mpl

# choose colors for different species
from matplotlib import cm
jet = cm.get_cmap('jet')
Number=15
jet_12_colors = jet(np.linspace(0, 1, Number))
color1=jet_12_colors[0,:]
color2=jet_12_colors[4,:]
color3=jet_12_colors[12,:]
color4=jet_12_colors[8,:]

out_filename = 'ToggleSwitch/ToggleSwitch_times1000_T40_merge' # Gillespie data path
data=np.load(out_filename+'.npz')
 
times = data['arr_0']   
time_points = data['arr_1']
Gx_total = data['arr_2']
Gx1_total = data['arr_3']
Gy_total = data['arr_4']
Gy1_total = data['arr_5']
Px_total = data['arr_6']
Py_total = data['arr_7']

#############################
AdaptiveT=0
Name='ToggleSwitch/Data-ToggleSwitch_1_32_M80_T8002_dt0.005_batch1000_epoch100' # VAN data path
data=np.load(str(Name)+'.npz', allow_pickle=True) 

argsSave = data['arr_1']
Tstep=argsSave[0]#1001
delta_t=argsSave[1]#0.05
L=argsSave[2]
print_step= argsSave[7]

SampleSum=data['arr_5']
delta_T= data['arr_6']
if AdaptiveT: TimePoins=np.cumsum(delta_T)[np.arange(SampleSum.shape[0])*print_step]
else: TimePoins=np.cumsum(delta_T)[np.arange(SampleSum.shape[0])*print_step]*delta_t

### 5.1 The time evolution of the average counts of species

In [ ]:
# The time evolution of the average counts for the genes and proteins,  from the VAN (dots) and the Gillespie simulation (lines). 
#####curve-----------------------------------
plt.rc('font', size=54)
markersize0=12
step=8

plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
plt.plot(time_points,np.mean(Gx_total,0),linewidth=6,color=color1,label='$G_x$')
plt.plot(TimePoins[::step],np.mean(SampleSum[:,:,0][::step],axis=1),
          marker='o',linestyle = 'None',color=color1,markersize=markersize0)

plt.plot(time_points,np.mean(Gy_total,0),linewidth=6,color=color2,label='$G_y$')
plt.plot(TimePoins[::step],np.mean(SampleSum[:,:,1][::step],axis=1),
          marker='o',linestyle = 'None',color=color2,markersize=markersize0)
plt.xlabel("Time (hr)")
# plt.title("Average Number")
plt.ylim([0,1.05])
plt.yticks([0,0.5,1])
fig.set_size_inches(9, 8)
plt.legend(numpoints=2,handletextpad=0.2,loc='lower right')
plt.title('Average Count',fontsize=64)
# plt.show()
plt.savefig('ToggleSwitch\ToggleSwitch_panela1.svg', bbox_inches="tight", dpi=400)

plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
plt.plot(time_points,np.mean(Py_total,0),linewidth=6,color=color4,label='$P_y$')
plt.plot(TimePoins[::step],np.mean(SampleSum[:,:,3][::step],axis=1),
          marker='o',linestyle = 'None',color=color4,markersize=markersize0)
plt.plot(time_points,np.mean(Px_total,0),linewidth=6,color=color3,label='$P_x$')
plt.plot(TimePoins[::step],np.mean(SampleSum[:,:,2][::step],axis=1),
          marker='o',linestyle = 'None',color=color3,markersize=markersize0)
plt.xlabel("Time (hr)")
plt.title("Average Count")
plt.ylim(top=42)
fig.set_size_inches(9, 8)
plt.legend(numpoints=2,handletextpad=0.2)
# plt.show()
plt.savefig('ToggleSwitch\ToggleSwitch_panela2.svg', bbox_inches="tight", dpi=400)

<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_panela1.svg" width="200"/> <br>
</td> 
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_panela2.svg" width="200"/> <br>
</td> 
</table>

### 5.2 Comparison of the means and standard deviations of the counts of species

In [ ]:
# Comparison of the means and standard deviations of the counts of the genes and proteins between the VAN and the Gillespie simulation, at time points t = 0, 1, ..., 39, 40.
####Gene-------------
plt.rc('font', size=56)
legendsize=50
T=40
NumTimePoints=1
markersize0=20
transparency=1-np.linspace(0, 0.99, int(T/NumTimePoints))
Range=[0,1]

plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
# Range=1#math.ceil(max(np.mean(Gx_total,0)))
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.mean(Gx_total[:,int(tt)],0),np.mean(SampleSum[:,:,0],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color1,markersize=markersize0,label='$G_x$')
    else:
        plt.plot(np.mean(Gx_total[:,int(tt)],0),np.mean(SampleSum[:,:,0],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color1,markersize=markersize0)
    k=k+1    
plt.plot(Range,Range,linewidth=4,color='black')

# Range=math.ceil(max(np.mean(Gy_total,0)))
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.mean(Gy_total[:,int(tt)],0),np.mean(SampleSum[:,:,1],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color2,markersize=markersize0,label='$G_y$')
    else:
        plt.plot(np.mean(Gy_total[:,int(tt)],0),np.mean(SampleSum[:,:,1],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color2,markersize=markersize0)
    k=k+1    
plt.plot(Range,Range,linewidth=4,color='black')


plt.xlabel('Gillespie')
plt.ylabel('VAN')
plt.title('Mean')
# plt.xticks([0.4,0.6,0.8])
# plt.yticks([0,30])
plt.legend(fontsize=legendsize,loc='best',handletextpad=0.2)
fig.set_size_inches(9, 8)
plt.savefig('ToggleSwitch\ToggleSwitch_panelb11.svg', bbox_inches="tight", dpi=400)


#####std----------------------------------
plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
Range=[0,1]
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.std(Gx_total[:,int(tt)],0),np.std(SampleSum[:,:,0],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color1,markersize=markersize0,label='$G_x$')
    else:
        plt.plot(np.std(Gx_total[:,int(tt)],0),np.std(SampleSum[:,:,0],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color1,markersize=markersize0)
    k=k+1    
plt.plot(Range,Range,linewidth=4,color='black')

k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.std(Gy_total[:,int(tt)],0),np.std(SampleSum[:,:,1],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color2,markersize=markersize0,label='$G_y$')
    else:
        plt.plot(np.std(Gy_total[:,int(tt)],0),np.std(SampleSum[:,:,1],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color2,markersize=markersize0)
    k=k+1    
plt.plot(Range,Range,linewidth=4,color='black')

plt.xlabel('Gillespie')
plt.ylabel('VAN')
plt.title('Std')

plt.legend(fontsize=legendsize,loc='best',handletextpad=0.2)
fig.set_size_inches(9, 8)
plt.savefig('ToggleSwitch\ToggleSwitch_panelb12.svg', bbox_inches="tight", dpi=400)

####Protein-------------
plt.rc('font', size=56)
legendsize=50
T=40
NumTimePoints=1
markersize0=20
transparency=1-np.linspace(0, 0.99, int(T/NumTimePoints))

plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
Range=50#math.ceil(max(np.mean(Py_total,0)))
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.mean(Py_total[:,int(tt)],0),np.mean(SampleSum[:,:,3],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color4,markersize=markersize0,label='$P_y$')
    else:
        plt.plot(np.mean(Py_total[:,int(tt)],0),np.mean(SampleSum[:,:,3],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color4,markersize=markersize0)
    k=k+1    
plt.plot(np.array([0,Range]),np.array([0,Range]),linewidth=4,color='black')

Range=math.ceil(max(np.mean(Px_total,0)))
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.mean(Px_total[:,int(tt)],0),np.mean(SampleSum[:,:,2],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color3,markersize=markersize0,label='$P_x$')
    else:
        plt.plot(np.mean(Px_total[:,int(tt)],0),np.mean(SampleSum[:,:,2],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color3,markersize=markersize0)
    k=k+1    
plt.plot(np.array([0,Range]),np.array([0,Range]),linewidth=4,color='black')


plt.xlabel('Gillespie')
plt.ylabel('VAN')
plt.title('Mean')
plt.legend(fontsize=legendsize,loc='best',handletextpad=0.2)
fig.set_size_inches(9, 8)
plt.savefig('ToggleSwitch\ToggleSwitch_panelb21.svg', bbox_inches="tight", dpi=400)

#####std----------------------------------
plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
Range=math.ceil(max(np.std(Py_total,0)))
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.std(Py_total[:,int(tt)],0),np.std(SampleSum[:,:,3],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color4,markersize=markersize0,label='$P_y$')
    else:
        plt.plot(np.std(Py_total[:,int(tt)],0),np.std(SampleSum[:,:,3],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color4,markersize=markersize0)
    k=k+1    
plt.plot(np.array([0,Range]),np.array([0,Range]),linewidth=4,color='black')

Range=25#.ceil(max(np.std(Px_total,0)))
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.std(Px_total[:,int(tt)],0),np.std(SampleSum[:,:,2],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color3,markersize=markersize0,label='$P_x$')
    else:
        plt.plot(np.std(Px_total[:,int(tt)],0),np.std(SampleSum[:,:,2],axis=1)[round(tt/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=color3,markersize=markersize0)
    k=k+1    
plt.plot(np.array([0,Range]),np.array([0,Range]),linewidth=4,color='black')

plt.xlabel('Gillespie')
plt.ylabel('VAN')
plt.title('Std')
plt.legend(fontsize=legendsize,loc='best',handletextpad=0.2)
fig.set_size_inches(9, 8)
plt.savefig('ToggleSwitch\ToggleSwitch_panelb22.svg', bbox_inches="tight", dpi=400)

<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_panelb11.svg" width="200"/> <br>
</td> 
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_panelb12.svg" width="200"/> <br>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_panelb21.svg" width="200"/> <br>
</td> 
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_panelb22.svg" width="200"/> <br>
</td> 
</table>

### 5.3 The marginal distributions of species

In [ ]:
# The marginal distributions of the gene Gx and protein Px at time point t=40 from the Gillespie simulation and the VAN. The inset shows the Hellinger distance between the two distributions.
#####Distribution---------------------------------------
plt.rc('font', size=38)
legendsize=30
# plt.rc('lines',linewidth=4)
T=40
Range1=3
Range2=8
Range3=8
axlinewidth=1.2

Gx1=Gx_total[:,int(T)]
Gx2=SampleSum[:,:,0][round(T/np.max(TimePoins)*TimePoins.shape[0])-1]
Gy1=Gy_total[:,int(T)]
Gy2=SampleSum[:,:,1][round(T/np.max(TimePoins)*TimePoins.shape[0])-1]
Px1=Px_total[:,int(T)]
Px2=SampleSum[:,:,2][round(T/np.max(TimePoins)*TimePoins.shape[0])-1]
Py1=Py_total[:,int(T)]
Py2=SampleSum[:,:,2][round(T/np.max(TimePoins)*TimePoins.shape[0])-1]
df_Gx1=pd.DataFrame(Gx1)
df_Gx2=pd.DataFrame(Gx2)
pro_Gx1=df_Gx1.value_counts(normalize=True).sort_index()
pro_Gx2=df_Gx2.value_counts(normalize=True).sort_index() 
df_Px1=pd.DataFrame(Px1)
df_Px2=pd.DataFrame(Px2)
pro_Px1=df_Px1.value_counts(normalize=True).sort_index()
pro_Px2=df_Px2.value_counts(normalize=True).sort_index()

HD_Gx=round(np.sqrt(1-np.sum(np.sqrt(pro_Gx1*pro_Gx2))),3)
HD_Px=round(np.sqrt(1-np.sum(np.sqrt(pro_Px1*pro_Px2))),3)

plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=4)
fig, axes = plt.subplots(1,1)
fig.tight_layout()
ax = plt.subplot(1,1, 1)
weights1 = np.ones_like(Gx1)/float(len(Gx1))
weights2 = np.ones_like(Gx2)/float(len(Gx2))
plt.hist([Gx1,Gx2],bins=Range1,weights=[weights1,weights2],color=['darkgrey',color1],alpha=0.7,orientation='horizontal')
plt.legend(['Gillespie','VAN'],title='$D_{HD}=$'+str(HD_Gx),title_fontsize=legendsize,fontsize=legendsize)
plt.ylabel("$G_x$")
plt.xlabel('Probability')
# plt.xticks([0,0.5])
plt.title("$t=$"+str(T))
plt.ylim(top=3)
fig.set_size_inches(9, 8)
plt.savefig('ToggleSwitch\ToggleSwitch_panelc1_T'+str(T)+'.svg', bbox_inches="tight", dpi=400)

plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=4)
fig, axes = plt.subplots(1,1)
fig.tight_layout()
ax = plt.subplot(1,1, 1)
weights1 = np.ones_like(Px1)/float(len(Px1))
weights2 = np.ones_like(Px2)/float(len(Px2))
plt.hist([Px1,Px2],bins=Range2,weights=[weights1,weights2],color=['darkgrey',color3],alpha=0.7,orientation='horizontal')
plt.legend(['Gillespie','VAN'],title='$D_{HD}=$'+str(HD_Px),title_fontsize=legendsize,fontsize=legendsize)
plt.ylabel("$P_x$")
plt.xlabel('Probability')
plt.title("$t=$"+str(T))
plt.ylim(top=123)
fig.set_size_inches(9, 8)
plt.savefig('ToggleSwitch\ToggleSwitch_panelc2_T'+str(T)+'.svg', bbox_inches="tight", dpi=400)

<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_panelc1_T40.svg" width="200"/> <br>
</td> 
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_panelc2_T40.svg" width="220"/> <br>
</td> 
</table>

### 5.4 The joint distribution of species

In [ ]:
#The joint distribution of the two proteins from the VAN at time point t=40, with the color bar for the probability values in the logarithmic scale.
###hist2d------------------------------------------------
cmpname='viridis'

plt.rc('font', size=38)

T0=40
T=round(T0/np.max(TimePoins)*TimePoins.shape[0])-1
bins=[35,35]#[int(max(SampleSum[T,:,2])),int(max(SampleSum[T,:,3]))]
plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=4)
fig, axes = plt.subplots(1,1)
fig.tight_layout()
ax = plt.subplot(1,1, 1,facecolor=[68/255,1/255,80/255])
h=plt.hist2d(SampleSum[T,:,2],SampleSum[T,:,3],bins=bins,cmap=cmpname,density=True,norm=mpl.colors.LogNorm(vmax=0.005,vmin=1e-5))
# plt.colorbar(label='Probability')
plt.xlabel("$P_x$")
plt.ylabel("$P_y$")
plt.title("$t=$"+str(T0))
plt.xlim(right=70)
plt.ylim(top=70)
fig.set_size_inches(9,8)
plt.savefig('ToggleSwitch\ToggleSwitch_paneld_T'+str(T0)+'.svg', bbox_inches="tight", dpi=400)
plt.show()

<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/ToggleSwitch_paneld_T40.svg" width="300"/> <br>
</td> 
</table>